## Preprocessing

In [18]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
drop_df = application_df.drop(columns= ["EIN", "NAME"])
drop_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [20]:
# Determine the number of unique values in each column.
drop_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [21]:
# Look at APPLICATION_TYPE value counts for binning
counts_hold = drop_df["APPLICATION_TYPE"].value_counts()
counts_hold

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [22]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(counts_hold[counts_hold<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [23]:
# Look at CLASSIFICATION value counts for binning
class_hold = drop_df["CLASSIFICATION"].value_counts()
class_hold

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [24]:
# You may find it helpful to look at CLASSIFICATION value counts >1
drop_df['CLASSIFICATION'].value_counts().loc[lambda x : x >1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [25]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_hold[class_hold<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [26]:
# Convert categorical data to numeric with `pd.get_dummies`
dummy = pd.get_dummies(drop_df)


In [27]:
# Split our preprocessed data into our features and target arrays
X = dummy.drop('IS_SUCCESSFUL', axis=1).values
y = dummy['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [28]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = X_train_scaled.shape[1]
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 30

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 80)                9360      
                                                                 
 dense_5 (Dense)             (None, 100)               8100      
                                                                 
 dense_6 (Dense)             (None, 30)                3030      
                                                                 
 dense_7 (Dense)             (None, 1)                 31        
                                                                 
Total params: 20,521
Trainable params: 20,521
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5750 - accuracy: 0.7145
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5543 - accuracy: 0.7310
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7305
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5495 - accuracy: 0.7288
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7310
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5459 - accuracy: 0.7329
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5454 - accuracy: 0.7324
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5444 - accuracy: 0.7326
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5438 - accuracy: 0.7333
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5426 - accura

804/804 [==============================] - 1s 2ms/step - loss: 0.5255 - accuracy: 0.7432
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5252 - accuracy: 0.7431
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5253 - accuracy: 0.7434
Epoch 84/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5254 - accuracy: 0.7428
Epoch 85/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5250 - accuracy: 0.7431
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5251 - accuracy: 0.7432
Epoch 87/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5248 - accuracy: 0.7437
Epoch 88/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5258 - accuracy: 0.7432
Epoch 89/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5246 - accuracy: 0.7433
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5249 - accuracy: 

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5745 - accuracy: 0.7293 - 347ms/epoch - 1ms/step
Loss: 0.5744546055793762, Accuracy: 0.7293294668197632


In [33]:
# Export our model to HDF5 file
nn_model.save('../h5_files/AlphabetSoupCharity_Optimization_v1.h5')